# Extract images at 432 * 768 resolution and 10 fps from mp4 videos

Requirements : 64 GB RAM, tqdm, imageio and moviepy

Runtime : 240 mins on aws i3.2xlarge instance

In [1]:
import imageio
imageio.plugins.ffmpeg.download()
import moviepy.editor as mpe
from glob import glob
import sys, os
import numpy as np
import scipy
%matplotlib inline
import matplotlib.pyplot as plt
import gc
from tqdm import tqdm
from scipy import sparse
from sklearn.utils.extmath import randomized_svd
import bcolz
import utils; reload(utils)
from utils import *
import scipy.signal as sci
from keras.preprocessing.image import ImageDataGenerator
import itertools


Imageio: 'ffmpeg.linux64' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg.linux64 (27.2 MB)
Downloading: 8192/28549024 bytes (0.0%)835584/28549024 bytes (2.9%)2326528/28549024 bytes (8.1%)4710400/28549024 bytes (16.5%)7831552/28549024 bytes (27.4%)10903552/28549024 bytes (38.2%)14057472/28549024 bytes (49.2%)17211392/28549024 bytes (60.3%)20340736/28549024 bytes (71.2%)23486464/28549024 bytes (82.3%)26566656/28549024 bytes (93.1%)28549024/28549024 bytes (100.0%)
  Done
File saved as /home/ubuntu/.imageio/ffmpeg/ffmpeg.linux64.


Using Theano backend.
WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: unknown error)


In [2]:
def create_data_matrix_from_video(clip,totalFrames,fps):
    return np.array([frame for frame,i in itertools.ifilter(lambda x: x[1]%fps==0,
                                                            zip(clip.iter_frames(),range(totalFrames)))])

In [3]:
def create_labels_for_video(path,name,totalFrames,fps):
    p = pd.read_csv(path + name + '.csv')
    return np.array([row for row,i in itertools.ifilter(lambda x: x[1]%fps==0,
                                                            zip(p.itertuples(),range(totalFrames)))])   

In [4]:
def processVideos(path,name):
    
    k=10
    video = mpe.VideoFileClip(path+name+'.mp4',target_resolution=(432,768))
    totalFrames = len([1 for frame in video.iter_frames()])
    fps=int(math.ceil(video.fps/k))
    
    RGB_HiRes = create_data_matrix_from_video(video, totalFrames,fps)
    labels = create_labels_for_video('../in/train/labels/',name,totalFrames,fps)
    
    save_array('../in/train/hires/'+name+'_hires.dat',RGB_HiRes)
    save_array('../in/train/labels/'+name+'_labels.dat',labels)
    
    del(RGB_HiRes)
    del(labels)
    gc.collect()

In [5]:
videos = glob('../in/train/mp4/*.mp4')   
totalDuration = 0
for i in range(len(videos)):
    name = videos[i].split("../in/train/mp4/")[1].split('.mp4')[0]
    video = mpe.VideoFileClip('../in/train/mp4/'+name+'.mp4')
    print video.duration, video.size, video.fps, name
    totalDuration += video.duration
print totalDuration

492.8 [1920, 1080] 29.18 train01
426.01 [1920, 1080] 29.18 train17
2434.16 [1920, 1080] 29.19 train19
739.11 [1920, 1080] 29.18 train12
940.93 [1920, 1080] 29.18 train08
437.93 [1920, 1080] 29.18 train05
876.18 [1920, 1080] 29.18 train10
474.28 [1920, 1080] 29.2 train06
480.45 [1920, 1080] 29.18 train13
478.01 [1920, 1080] 29.18 train18
529.72 [1920, 1080] 29.19 train21
1398.14 [1920, 1080] 29.19 train25
624.76 [1920, 1080] 29.18 train16
580.12 [1920, 1080] 29.2 train20
707.93 [1920, 1080] 29.19 train22
547.14 [1920, 1080] 29.19 train09
488.38 [1920, 1080] 29.18 train11
383.05 [1920, 1080] 29.19 train23
599.29 [1920, 1080] 29.18 train03
637.03 [1920, 1080] 29.18 train24
703.84 [1920, 1080] 29.19 train14
471.53 [1920, 1080] 29.18 train15
454.52 [1920, 1080] 29.18 train04
642.82 [1920, 1080] 29.19 train07
405.45 [1920, 1080] 29.19 train02
16953.58


In [ ]:
files = ['train01','train02','train03','train04','train05','train06','train07','train08','train09','train10','train11',
         'train12','train13','train14','train15','train16','train17','train18','train19','train20','train21','train22',
         'train23','train24' , 'train25'] 

for i in tqdm(range(len(files))):
    processVideos('../in/train/mp4/',files[i])

 12%|█▏        | 3/25 [13:24<1:36:11, 262.32s/it]

# Create Validation dataset (i.e. 360 x 640 resolution @ 10 fps)
videos 03, 04, 10, 18, 21 - 100%

In [4]:
def center_crop(x, random_crop_size, mode = ''):
    #mode can be center (blanks), top_left, top_right, bottom_left or bottom_right
    w, h = x.shape[1], x.shape[2] 
    rangew = (w - random_crop_size[0]) // 2 
    rangeh = (h - random_crop_size[1]) // 2 
    if mode=='top_left':
        rangew = 0 
        rangeh = 0 
    if mode=='top_right':
        rangew = 0
        rangeh = (w - random_crop_size[0]) 
    if mode=='bottom_left':
        rangew = (w - random_crop_size[0])
        rangeh = 0 
    if mode=='bottom_right':
        rangew = (w - random_crop_size[0])
        rangeh = (w - random_crop_size[0])
    offsetw = 0 if rangew == 0 else rangew
    offseth = 0 if rangeh == 0 else rangeh
    return x[:, offsetw:offsetw+random_crop_size[0], offseth:offseth+random_crop_size[1]]

In [5]:
def resizeVideos(mode):
    files = ['train03','train04','train10','train18','train21']
    for i in range(len(files)):
        RGB_HiRes = load_array('../in/train/hires/'+files[i]+'_hires.dat')
        labels = load_array('../in/train/labels/'+files[i]+'_labels.dat')
        totalFrames = labels.shape[0]
        rgbResized = np.zeros([totalFrames, 360, 640 , 3 ],dtype=np.uint8) #360,640
        rgbResized = center_crop(RGB_HiRes, (360,640),mode)#360,640
        save_array('../in/train/hires/'+files[i]+mode+'_val_hires.dat',rgbResized)
        save_array('../in/train/labels/'+files[i]+'_val_labels.dat',labels)
        del(RGB_HiRes,labels,rgbResized)
        gc.collect() 

videos 14,19 and 24 - Only 20% of frames with Vannas.scissors, Mendez.ring, vitrectomy.handpiece, needle.holder or biomarker

In [6]:
def extractValidationFrames(mode):
    
    files = ['train14','train19','train24']
    for i in range(len(files)):
        RGB_HiRes = load_array('../in/train/hires/'+files[i]+'_hires.dat')
        labels = load_array('../in/train/labels/'+files[i]+'_labels.dat')
        totalFrames = labels.shape[0]
        rgbResized = np.zeros([25000, 360, 640 , 3 ],dtype=np.uint8) # <== Hardcoded to 1000 to save memory
        rgbResizedLabels = np.zeros([totalFrames, 23 ],dtype=np.int8)
        rgbMinusVal = np.zeros([totalFrames, 432, 768 , 3 ],dtype=np.uint8) #432,768
        rgbMinusValLabels = np.zeros([totalFrames, 23 ],dtype=np.int8)
        rgbResizedIndex = 0
        rgbMinusValIndex = 0
        Label_0_FramesToKeepInValidation = 6
        Label_1_FramesToKeepInValidation = 1822
        Label_10_FramesToKeepInValidation = 15
        Label_13_FramesToKeepInValidation = 912
        Label_19_FramesToKeepInValidation = 35
        Label_20_FramesToKeepInValidation = 15

        for j in range(totalFrames):
            validation = False       
            if (labels[j,2]==1) and (Label_0_FramesToKeepInValidation>0):
                Label_0_FramesToKeepInValidation -= 1
                validation = True
            if (labels[j,3]==1) and (Label_1_FramesToKeepInValidation>0):
                Label_1_FramesToKeepInValidation -= 1
                validation = True 
            if (labels[j,12]==1) and (Label_10_FramesToKeepInValidation>0):
                Label_10_FramesToKeepInValidation -= 1
                validation = True    
            if (labels[j,15]==1) and (Label_13_FramesToKeepInValidation>0):
                Label_13_FramesToKeepInValidation -= 1
                validation = True    
            if (labels[j,21]==1) and (Label_19_FramesToKeepInValidation>0):
                Label_19_FramesToKeepInValidation -= 1
                validation = True    
            if (labels[j,22]==1) and (Label_20_FramesToKeepInValidation>0):
                Label_20_FramesToKeepInValidation -= 1
                validation = True                
            if validation ==True :
                rgbResized[rgbResizedIndex] = center_crop(np.expand_dims(RGB_HiRes[j],0), (360,640),mode)[0] #360,640
                rgbResizedLabels[rgbResizedIndex] = labels[j]
                rgbResizedIndex += 1
            else:
                if mode=='':
                    rgbMinusVal[rgbMinusValIndex] = RGB_HiRes[j]
                    rgbMinusValLabels[rgbMinusValIndex] = labels[j]
                    rgbMinusValIndex += 1

        save_array('../in/train/labels/'+files[i]+'_val_labels.dat',rgbResizedLabels[0:rgbResizedIndex])
        save_array('../in/train/hires/'+files[i]+mode+'_val_hires.dat',rgbResized[0:rgbResizedIndex])
        if mode=='':
            save_array('../in/train/labels/'+files[i]+'_minus_val_labels.dat',rgbMinusValLabels[0:rgbMinusValIndex])
            save_array('../in/train/hires/'+files[i]+'_minus_val_hires.dat',rgbMinusVal[0:rgbMinusValIndex])
        print rgbResizedLabels[0:rgbResizedIndex].shape, rgbResized[0:rgbResizedIndex].shape,rgbMinusValLabels[0:rgbMinusValIndex].shape, rgbMinusVal[0:rgbMinusValIndex].shape
        del(RGB_HiRes,labels,rgbResized,rgbResizedLabels,rgbMinusVal,rgbMinusValLabels)
        gc.collect() 

Remove 0.5 labeled frames from validation dataset

In [7]:
def remove0_5(mode):
    
    files = ['train03','train04','train10','train18','train21', 'train14','train19','train24']

    for i in range(len(files)):
        RGB_HiRes = load_array('../in/train/hires/'+files[i]+mode+'_val_hires.dat')
        labels = load_array('../in/train/labels/'+files[i]+'_val_labels.dat')
        totalFrames = labels.shape[0]
        rgbNew = np.zeros([totalFrames, 360, 640 , 3 ],dtype=np.uint8)
        labelNew = np.zeros([totalFrames, 23 ],dtype=np.int8)
        index = 0
        for frameIndex in range(totalFrames):
            if labels[frameIndex,2]==0.5 or labels[frameIndex,3]==0.5 or labels[frameIndex,4]==0.5 or labels[frameIndex,5]==0.5 or labels[frameIndex,6]==0.5 or labels[frameIndex,7]==0.5 or labels[frameIndex,8]==0.5 or labels[frameIndex,9]==0.5 or labels[frameIndex,10]==0.5 or labels[frameIndex,11]==0.5 or labels[frameIndex,12]==0.5 or labels[frameIndex,13]==0.5 or labels[frameIndex,14]==0.5 or labels[frameIndex,15]==0.5 or labels[frameIndex,16]==0.5 or labels[frameIndex,17]==0.5 or labels[frameIndex,6]==0.5 or labels[frameIndex,18]==0.5 or labels[frameIndex,19]==0.5 or labels[frameIndex,20]==0.5 or labels[frameIndex,21]==0.5 or labels[frameIndex,22]==0.5:
                 index += 0
            else:
                rgbNew[index] = RGB_HiRes[frameIndex]
                labelNew[index] = labels[frameIndex]
                index += 1
        save_array('../in/train/hires/'+files[i]+mode+'_val_minus_0p5_hires.dat',rgbNew[0:index])
        save_array('../in/train/labels/'+files[i]+'_val_minus_0p5_labels.dat',labelNew[0:index])
        print rgbNew[0:index].shape, labelNew[0:index].shape, RGB_HiRes.shape, labels.shape, index
        del(RGB_HiRes,labels,rgbNew,labelNew)

In [8]:
def generateTestTimeAugmentationPatches(mode):
    resizeVideos(mode)
    extractValidationFrames(mode)
    remove0_5(mode)

In [9]:
generateTestTimeAugmentationPatches('')
generateTestTimeAugmentationPatches('top_left')
generateTestTimeAugmentationPatches('top_right')
generateTestTimeAugmentationPatches('bottom_left')
generateTestTimeAugmentationPatches('bottom_right')

(83, 23) (83, 360, 640, 3) (0, 23) (0, 432, 768, 3)
(2734, 23) (2734, 360, 640, 3) (0, 23) (0, 432, 768, 3)
(41, 23) (41, 360, 640, 3) (0, 23) (0, 432, 768, 3)
(5695, 360, 640, 3) (5695, 23) (5830, 360, 640, 3) (5830, 23) 5695
(4384, 360, 640, 3) (4384, 23) (4421, 360, 640, 3) (4421, 23) 4384
(8389, 360, 640, 3) (8389, 23) (8523, 360, 640, 3) (8523, 23) 8389
(4565, 360, 640, 3) (4565, 23) (4650, 360, 640, 3) (4650, 23) 4565
(4884, 360, 640, 3) (4884, 23) (5155, 360, 640, 3) (5155, 23) 4884
(83, 360, 640, 3) (83, 23) (83, 360, 640, 3) (83, 23) 83
(2734, 360, 640, 3) (2734, 23) (2734, 360, 640, 3) (2734, 23) 2734
(41, 360, 640, 3) (41, 23) (41, 360, 640, 3) (41, 23) 41


# Mark unwanted frames

In [10]:
files = ['train01','train02','train05','train06','train07','train08','train09','train11',
         'train12','train13','train14_minus_val','train15','train16','train17','train19_minus_val','train20',
         'train22','train23','train24_minus_val','train25'] 

labels = load_array('/cat/home/ubuntu/cat/in/train/labels/train01_labels.dat')
for i in range(1,len(files)):
    lbl = load_array('/cat/home/ubuntu/cat/in/train/labels/'+files[i]+'_labels.dat')
    labels = np.vstack((labels,lbl))
    del(lbl)
    gc.collect()
#labels[labels==0.5]=999
anyLabel = sum(np.transpose(labels)[3:23,:])
from scipy.ndimage.interpolation import shift

lagggedLabels = shift(anyLabel, -15, cval=1)
lagggedLabels[lagggedLabels<0.5]=0
for i in range(30):
    temp = shift(anyLabel, -14+i, cval=1)
    temp[temp<0.5]=0
    lagggedLabels = np.vstack((lagggedLabels,temp))

anyLabelLag15 = sum(lagggedLabels)
anyLabelLag15[anyLabelLag15>0]=1
keepOrNot = np.vstack((anyLabel,anyLabelLag15)).T
unique, counts = np.unique(keepOrNot[:,0], return_counts=True)
unique, counts

(array([ 0. ,  0.5,  1. ,  1.5,  2. ,  2.5,  3. ]),
 array([62706,  2108, 46971,   325, 21299,    44,    57]))

In [11]:
unique, counts = np.unique(keepOrNot[:,1], return_counts=True)
unique, counts

(array([ 0.,  1.]), array([48302, 85208]))

In [12]:
keepOrNot.shape

(133510, 2)

In [13]:
#QC
videos = glob('/cat/home/ubuntu/cat/in/train/mp4/*.mp4')
path='/cat/home/ubuntu/cat/in/train/labels/'
totalFrames = 0
totalFrames_Array = 0
for i in range(len(videos)):
    name = videos[i].split("/cat/home/ubuntu/cat/in/train/mp4/")[1].split('.mp4')[0]
    labels_file = pd.read_csv(path + name + '.csv').values
    labels_array = load_array('/cat/home/ubuntu/cat/in/train/labels/'+name+'_labels.dat')
    print name, labels_file.shape[0] , labels_array.shape[0]
    totalFrames += labels_file.shape[0]
    totalFrames_Array += labels_array.shape[0]
print totalFrames,totalFrames/3, totalFrames_Array

train01 14380 4794
train17 12433 4144
train19 71066 23685
train12 21574 7190
train08 27463 9153
train05 12778 4260
train10 25569 8523
train06 13848 4616
train13 14021 4674
train18 13952 4650
train21 15463 5155
train25 40820 13604
train16 18232 6077
train20 16939 5647
train22 20664 6888
train09 15973 5324
train11 14252 4751
train23 11182 3728
train03 17490 5830
train24 18591 6197
train14 20544 6848
train15 13762 4587
train04 13266 4421
train07 18769 6255
train02 11837 3946
494868 164956 164947


In [14]:
keepFileindex=0
arrayIndex = 0
FIRST_TIME = True
for j in range(keepOrNot.shape[0]):
    FringeFrame = 1
    if(keepOrNot[j,0]<4):
        if((keepOrNot[j,1]==0 and keepOrNot[j,0]==0) or keepOrNot[j,0]>0):
            FringeFrame = 0
    labels[j,1] = FringeFrame

In [15]:
files = ['train01','train02','train05','train06','train07','train08','train09','train11',
         'train12','train13','train14_minus_val','train15','train16','train17','train19_minus_val','train20',
         'train22','train23','train24_minus_val','train25'] 
index = 0
for i in range(len(files)):
    lbl = load_array('/cat/home/ubuntu/cat/in/train/labels/'+files[i]+'_labels.dat')
    save_array('../in/train/labels/'+files[i]+'_aug_labels.dat',labels[index:index+lbl.shape[0]])
    index += lbl.shape[0]
    print index

4794
8740
13000
17616
23871
33024
38348
43099
50289
54963
61728
66315
72392
76536
97487
103134
110022
113750
119906
133510


## QC

In [16]:
files = ['train01','train02','train05','train06','train07','train08','train09','train11',
         'train12','train13','train14_minus_val','train15','train16','train17','train19_minus_val','train20',
         'train22','train23','train24_minus_val','train25'] 
for i in range(len(files)):
    lbl = load_array('/cat/home/ubuntu/cat/in/train/labels/'+files[i]+'_labels.dat')
    lbl2 = load_array('/cat/home/ubuntu/cat/in/train/labels/'+files[i]+'_aug_labels.dat')
    print lbl.shape, lbl2.shape

(4794, 23) (4794, 23)
(3946, 23) (3946, 23)
(4260, 23) (4260, 23)
(4616, 23) (4616, 23)
(6255, 23) (6255, 23)
(9153, 23) (9153, 23)
(5324, 23) (5324, 23)
(4751, 23) (4751, 23)
(7190, 23) (7190, 23)
(4674, 23) (4674, 23)
(6765, 23) (6765, 23)
(4587, 23) (4587, 23)
(6077, 23) (6077, 23)
(4144, 23) (4144, 23)
(20951, 23) (20951, 23)
(5647, 23) (5647, 23)
(6888, 23) (6888, 23)
(3728, 23) (3728, 23)
(6156, 23) (6156, 23)
(13604, 23) (13604, 23)


In [17]:
lbl[3000]

array([ 9000.,  9001.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.])

In [18]:
lbl2[3000]

array([ 9000.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.])

In [19]:
print np.sum(lbl[:,20]), np.sum(lbl2[:,20])

50.0 50.0
